In [297]:
import requests
import pandas as pd
import numpy as np
import json
# Progress bars
import tqdm
from tqdm import tqdm_notebook
import lzma
from multiprocessing import Pool
import os

from collections import OrderedDict, defaultdict
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    #'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

In [178]:
with open('../v3_json_data/genrebiglabels.json') as f:
    genres_list = json.load(f)

# Genres and starting points:
genres = [(feature['properties']['name'], feature['geometry']['coordinates']) for feature in genres_list['features']]

In [179]:
genres[4][0].lower().replace(' ', '')

'dirtysouthrap'

In [180]:
genres

[('Rap', [8330, 18105]),
 ('Ghetto Tech', [14930, 19205]),
 ('Technobass', [12230, 19105]),
 ('Miami Bass', [11930, 18905]),
 ('Dirty South Rap', [12230, 18805]),
 ('Crunk', [15230, 18705]),
 ('Moombahton', [19130, 19005]),
 ('Trap', [19430, 18605]),
 ('Southern Rap', [12530, 18505]),
 ('Westcoast Rap', [12830, 18405]),
 ('Eastcoast Rap', [12230, 18205]),
 ('Bling', [16430, 18305]),
 ('Dancehall', [11930, 18005]),
 ('Conscious Rap', [13130, 17905]),
 ('Themed Rap', [14030, 17805]),
 ('Turntablism', [11030, 17705]),
 ('Freestyle', [11330, 17505]),
 ('Florida Breaks', [14930, 17405]),
 ('Electro', [10730, 17305]),
 ('Minimal Techno', [14030, 17005]),
 ('Dub Techno', [14330, 16905]),
 ('Bangin Techno', [14930, 16805]),
 ('Detroit Techno', [10730, 16705]),
 ('Bleep Techno', [13130, 16605]),
 ('Hard Techno', [14330, 16505]),
 ('Euro Techno', [13130, 16205]),
 ('Schranz', [16430, 16405]),
 ('Experimental Techno', [16130, 16305]),
 ('Speedcore', [16430, 15905]),
 ('Rave', [13430, 16005]),
 ('

In [58]:
# Tracklists: http://music.ishkur.com/musicbox.php?x=9540&y=7685&match=genre
# Descriptions: http://music.ishkur.com/content.php?g=Microfunk

In [362]:
def save_genre(params):
    name = params[0].replace(' ', '').replace('/','%2F')
    desc_name = params[0].replace(' ','%20').replace('/','%2F')
    
    x = params[1][0]
    y = params[1][1]
    
    desc_fname = "../v3_html_data/{}_description.html.xz".format(name.lower())
    desc_url = "http://music.ishkur.com/content.php?g={}".format(desc_name)
    cache_url(desc_url, desc_fname, overwrite=True) #
    
    tracks_fname = "../v3_html_data/{}_tracks.html.xz".format(name.lower())
    tracks_url = "http://music.ishkur.com/musicbox.php?x={}&y={}&match=genre".format(x,y)
    cache_url(tracks_url, tracks_fname)

# Save a given url as a file
def cache_url(url, fname, overwrite=False):
    if os.path.isfile(fname) and not overwrite:
        return

    try: 
        data = requests.get(url, stream=False, timeout=30, allow_redirects=True, headers=headers).content
    except Exception as e:
        print("error:", url)
        return
    
    with lzma.open(fname, 'wb') as f:            
        f.write(data)

# Show Progress bar and perform a function on an iterable in parallel:
def process_with_progress(func, iterable):
    with Pool(8) as p:
        results = list(tqdm_notebook(p.imap(func, iterable), total=len(iterable)))
    return results

In [363]:
save_genre(genres[0])

In [364]:
process_with_progress(save_genre, genres)
print()

In [185]:
# Derive Scene based on position:
#def y_sort(item):
#    return item['geometry']['coordinates'][1]

#with open('../v3_json_data/scenelabels.json') as f:
#    scenes = json.load(f)

#iter_scenes = iter(sorted(scenes['features'], key=y_sort))#
#
#first = next(iter_scenes)
#y = first['geometry']['coordinates'][1]
#name = first['properties']['name']#
#
#scene_dict = {}
#for scene in iter_scenes:#
#
#    next_y = scene['geometry']['coordinates'][1]
#    next_name = scene['properties']['name']
#    
#    scene_dict[range(y, next_y)] = name#
#
#    y = next_y
#    name = next_name#
#
#scene_dict

In [186]:
#with open('../v3_json_data/genrebiglabels.json') as f:
#    genres = json.load(f)
#
#_ = [print(json.dumps(s)) for s in sorted(genres['features'], key=y_sort)]
# Manually split - genre big label y coords not helpful, ovelap with labels

In [187]:
#sorted(scenes['features'], key=y_sort)

In [188]:
#[scene_dict[k] for k in scene_dict.keys() if 1305 in k]

In [189]:
#{ "type": "Feature", "properties": { "name": "Trap"}, "geometry": { "type": "Point", "coordinates": [19430, 18605] } }, #bass
#{ "type": "Feature", "properties": { "name": "Southern Rap"}, "geometry": { "type": "Point", "coordinates": [12530, 18505] } }, #hip hop

In [190]:
def y_sort(item):
    return item['geometry']['coordinates'][1]

with open('../v3_json_data/sorted_genres.jsonl') as f:
    # Manually split by newlines
    groups_lines = f.read().split('\n\n')    
    groups = [[json.loads(g) for g in group.split('\n') if g is not ''] for group in groups_lines]
    print(len(groups))
    
with open('../v3_json_data/scenelabels.json') as f:
    scenes = sorted(json.load(f)['features'], key=y_sort)
    scenes = [scene['properties']['name'] for scene in scenes]
    # instert extra pioneers to cover minimalism etc. (spans across IDM)
    scenes.remove('Pioneers')
    scenes.insert(9, 'Pioneers')
    scenes.insert(10, 'Intelligent Dance Music')
    scenes.insert(12,'Pioneers')
    scenes.insert(15,'Pioneers')
    print(len(scenes))

31
31


In [196]:
groups[8]

[{'type': 'Feature',
  'properties': {'name': 'Breakcore'},
  'geometry': {'type': 'Point', 'coordinates': [17030, 5605]}},
 {'type': 'Feature',
  'properties': {'name': 'Drill n Bass'},
  'geometry': {'type': 'Point', 'coordinates': [14330, 5705]}},
 {'type': 'Feature',
  'properties': {'name': 'Glitch Hop'},
  'geometry': {'type': 'Point', 'coordinates': [17030, 5805]}},
 {'type': 'Feature',
  'properties': {'name': 'Glitch'},
  'geometry': {'type': 'Point', 'coordinates': [16430, 5905]}}]

In [197]:
scenes[8]

'Intelligent Dance Music'

In [219]:
# oops. overwrites, multiple entries..
#scene_to_genre = OrderedDict(zip(scenes, [[g['properties']['name'] for g in group] for group in groups]))

scene_to_genre = OrderedDict()
for scene,group_names in zip(scenes, [[g['properties']['name'] for g in group] for group in groups]):
    #print (scene, group_names)
    if scene not in scene_to_genre:
        scene_to_genre[scene] = group_names
    else:
        g_list = scene_to_genre[scene]
        g_list.extend(group_names)
        scene_to_genre[scene] = g_list

genre_to_scene = OrderedDict((i,k) for k,v in scene_to_genre.items() for i in v)
genre_to_scene

OrderedDict([('Dancepunk', 'Eurotrash'),
             ('Filthy Electrohouse', 'Eurotrash'),
             ('Electroclash', 'Eurotrash'),
             ('Hi NRG', 'Eurodisco'),
             ('Synthpop', 'Eurodisco'),
             ('Synthwave', 'Eurodisco'),
             ('Spacesynth', 'Eurodisco'),
             ('Italo Disco', 'Eurodisco'),
             ('Eurobeat', 'Europop'),
             ('Asian Pop', 'Europop'),
             ('Nu Italo', 'Europop'),
             ('Eurodance', 'Europop'),
             ('Handsup', 'Europop'),
             ('Vocal Trance', 'Europop'),
             ('Dutch House', 'Eurotrance'),
             ('Anthem House', 'Eurotrance'),
             ('Eurotrance', 'Eurotrance'),
             ('German Trance', 'Eurotrance'),
             ('Hard Trance', 'Hard Dance'),
             ('Hardstyle', 'Hard Dance'),
             ('Jumpstyle', 'Hard Dance'),
             ('UK Hardhouse', 'Hard Dance'),
             ('NRG', 'Hard Dance'),
             ('Tech Trance', 'Trance'),


In [221]:
#scene_to_genre['Industrial/Goth']

In [222]:
#[[(k,i) for i in v] for k,v in scene_to_genre.items()]

In [223]:
#with open('../v3_json_data/coords.json') as f:
#    coords = json.load(f)['features']

In [224]:
#coords[140]['geometry']['coordinates'][0]

In [225]:
#coords[164]

In [226]:
# Coords for years: 2018 start: 21900, ever 300 is a year.
# coords from gcpoly: [[ [22200, 6125], [22200, 6085], [21900, 6085], [21900, 6125], [22200, 6125] ]]
                    #       right  Top    right bottom  left   bottom   left top       right top

In [276]:
# typo: eastcost / eastcoast rap; LIquid / Liquid funk; US Deep House (1992) Mr. Fingers'; 

with open('../v3_json_data/gcpoly.json') as f:
    coords = json.load(f)['features']

In [277]:
# x values to years:
x_to_year = dict(zip(range(22200, 0, -300), range(2019,1946,-1)))

In [278]:
def extract(coord):
    # Index into left coord x
    query_point = sorted(coord['geometry']['coordinates'][0])[0]
    return {"name":coord['properties']['name'], "tracks": coord['properties']['value'], "year":x_to_year[query_point[0]], "query": query_point}


In [279]:
all_years = [extract(coord) for coord in coords] 

In [280]:
len(all_years)

2856

In [281]:
all_years[2]

{'name': '2-Step Garage', 'tracks': '12', 'year': 2000, 'query': [16500, 9485]}

In [282]:
# scrape all the years tracks - heavy handed, but easier than parsing track names and finding years from one big list

#
    
def save_track_year(params):
    name = params['name'].replace(' ', '').replace('/','%2F').lower()
    
    x = params['query'][0]
    y = params['query'][1]
    
    tracks_fname = "../v3_html_data/{}_{}_tracks.html.xz".format(name,params['year'])    
    tracks_url = "http://music.ishkur.com/musicbox.php?x={}&y={}&match=year".format(x,y)
    cache_url(tracks_url, tracks_fname)

In [283]:
process_with_progress(save_track_year, all_years)
print()

In [284]:
genre_first_year = defaultdict()

for entry in all_years:
    year = entry['year']
    name = entry['name']
    
    if name not in genre_first_year:
        genre_first_year[name] = year
    
    if genre_first_year[name] > year:
        genre_first_year[name] = year

In [285]:
genre_first_year

defaultdict(None,
            {'2-Step Garage': 1998,
             '8th Note Prog': 2007,
             'Acid House': 1986,
             'Acid Jazz': 1986,
             'Acid': 1988,
             'Aggrotech': 1997,
             'Ambient Techno': 1990,
             'Ambient': 1977,
             'Amiga/Tracker': 1985,
             'Anthem House': 1991,
             'Asian Pop': 1991,
             'Atmospheric Jungle': 1993,
             'Balearic Trance': 1991,
             'Bangin Techno': 1995,
             'Big Beat': 1995,
             'Bleep Techno': 1989,
             'Bling': 2000,
             'Braindance': 1993,
             'Breakcore': 2002,
             'Breaks': 1987,
             'Brostep': 2008,
             'Chemical Breaks': 1993,
             'Chicago House': 1984,
             'Chill Out': 1977,
             'Chiptune': 1980,
             'Collage': 1956,
             'Conscious Rap': 1989,
             'Crunk': 1996,
             'Dancehall': 1985,
             'Dancep

In [286]:
# for the network:

def clean_name(name):
    return name.replace(' ', '').replace('/','%2F').lower()

def year_start(name):
    return genre_first_year[name]

def self_node(name):
    return "{},{},{},".format(clean_name(name), clean_name(name), year_start(name))


print("source,target,start,end")
[print(self_node(name)+',\n'+clean_name(name)+',' )  for name in reversed(genre_to_scene.keys())]
print()

source,target,start,end
ghettotech,ghettotech,1995,,
ghettotech,
technobass,technobass,1986,,
technobass,
moombahton,moombahton,2009,,
moombahton,
miamibass,miamibass,1985,,
miamibass,
dirtysouthrap,dirtysouthrap,1986,,
dirtysouthrap,
crunk,crunk,1996,,
crunk,
trap,trap,2010,,
trap,
southernrap,southernrap,1987,,
southernrap,
westcoastrap,westcoastrap,1988,,
westcoastrap,
bling,bling,2000,,
bling,
eastcoastrap,eastcoastrap,1988,,
eastcoastrap,
rap,rap,1979,,
rap,
dancehall,dancehall,1985,,
dancehall,
consciousrap,consciousrap,1989,,
consciousrap,
themedrap,themedrap,1992,,
themedrap,
turntablism,turntablism,1982,,
turntablism,
freestyle,freestyle,1983,,
freestyle,
floridabreaks,floridabreaks,1995,,
floridabreaks,
electro,electro,1981,,
electro,
minimaltechno,minimaltechno,1992,,
minimaltechno,
dubtechno,dubtechno,1993,,
dubtechno,
bangintechno,bangintechno,1995,,
bangintechno,
detroittechno,detroittechno,1981,,
detroittechno,
bleeptechno,bleeptechno,1989,,
bleeptechno,
hardtechno,hardt

In [287]:
#

In [288]:
all_years

[{'name': '2-Step Garage',
  'tracks': '4',
  'year': 1998,
  'query': [15900, 9485]},
 {'name': '2-Step Garage',
  'tracks': '10',
  'year': 1999,
  'query': [16200, 9485]},
 {'name': '2-Step Garage',
  'tracks': '12',
  'year': 2000,
  'query': [16500, 9485]},
 {'name': '2-Step Garage',
  'tracks': '4',
  'year': 2001,
  'query': [16800, 9485]},
 {'name': '2-Step Garage',
  'tracks': '4',
  'year': 2002,
  'query': [17100, 9485]},
 {'name': '2-Step Garage',
  'tracks': '1',
  'year': 2003,
  'query': [17400, 9485]},
 {'name': '2-Step Garage',
  'tracks': '1',
  'year': 2010,
  'query': [19500, 9485]},
 {'name': '8th Note Prog',
  'tracks': '3',
  'year': 2007,
  'query': [18600, 10885]},
 {'name': '8th Note Prog',
  'tracks': '11',
  'year': 2008,
  'query': [18900, 10885]},
 {'name': '8th Note Prog',
  'tracks': '2',
  'year': 2010,
  'query': [19500, 10885]},
 {'name': 'Acid House', 'tracks': '3', 'year': 1986, 'query': [12300, 12985]},
 {'name': 'Acid House', 'tracks': '7', 'year'

In [305]:
fname = "../v3_html_data/{}_{}_tracks.html.xz".format("2-stepgarage", 1998)
with lzma.open(fname, 'rb') as f:            
    soup = BeautifulSoup(f.read(), "lxml")
    print()
    for source in soup.select('source'):
        parts = source['title'].split(' - ')
        print(parts)


['Groove Chronicles', 'Stone Cold']
['M-Dubs', 'Over Here (Sugar Shack Breakbeat Funk)']
['Antonio', 'Hyperfunk']
['Box Clever', 'Talk To Me']


In [357]:
#track_metadata = [(entry['scene'] = genre_to_scene[entry['name']]) for entry in all_years]

df =pd.DataFrame()

def make_tracks(entry):
    entry['scene'] = genre_to_scene[entry['name']]
    entry['slug'] = clean_name(entry['name'])

    
    tracks = []
    
    fname = "../v3_html_data/{}_{}_tracks.html.xz".format(entry['slug'], entry['year'])
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        
        for source in soup.select('source'):
            entry = entry.copy()
            parts = source['title'].split(' - ')
            entry['artist'] = parts[0]
            entry['title'] = parts[1]
            
            tracks.append(entry)
    
    return tracks
    
df = pd.DataFrame([track for entry in all_years for track in make_tracks(entry)], dtype=str)
df.rename({'name': 'genre'}, axis=1, inplace=True)
df = df[['scene', 'genre', 'year', 'artist', 'title']]

In [358]:
df.sample(10)

,scene,genre,year,artist,title
5538,Garage/Deep House,Garage,1996,Deep Bros feat Michael Watford,You Got It (Tuff Jam mix 1)
10740,Progressive,Twinkle Prog,2004,Synergy,Pacifier (David West & Inkfish De Nada mix)
8967,Progressive,Progressive,2006,Max Graham,Space Disco
10131,Eurodisco,Synthwave,2012,Unit Black Flight,Shadow Scope
4040,Garage/Deep House,Euro Deep House,1991,MBG,Ore Nove Nove
2702,Tech House,Deeptech,2014,Noir & Hayze,Angel (Hot Since 82 Vocal mix)
2024,Hip Hop,Conscious Rap,1994,Arrested Developmet,Warm Sentiments
6756,Chill Out,Indie,2010,Gorillaz,Empire Ants
1770,Chill Out,Chill Out,1998,GusGus,Anthem
2715,Tech House,Deeptech,2015,Return Of The Jaded,Feel Ya


In [359]:
df.describe(include='all')

,scene,genre,year,artist,title
count,11317,11317,11317,11317,11317
unique,28,166,65,9594,10724
top,House,Synthpop,2000,Eat Static,Don't Stop
freq,812,256,445,6,6


In [360]:
df.to_csv('../v3_tracks.csv',index=False)

In [361]:
# Descriptions:

In [384]:
#genre_to_scene.keys()

In [380]:
def get_description(genre):
    fname = "../v3_html_data/{}_description.html.xz".format(genre)
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        genre_name = soup.select('td.header', limit=1)[0].text
        genre_aka = soup.select('td.miniheader', limit=1)[0].text
        genre_scene = soup.select('td.subheader', limit=1)[0].text
        genre_emerge = soup.select('td.miniheader')[-1].text
    return {'name': genre_name,
           'aka':genre_aka,
           'scene':genre_scene,
           'emerged':genre_emerge}

In [387]:
df_genres = pd.DataFrame([get_description(clean_name(genre)) for genre in genre_to_scene.keys()])
df_genres['slug'] = df_genres['name'].apply(clean_name)
df_genres.rename({'name': 'genre'}, axis=1, inplace=True)
df_genres = df_genres[['scene', 'genre', 'emerged', 'aka', 'slug']]
df_genres.sample(10)

,scene,genre,emerged,aka,slug
129,Drum n Bass,Jazzstep,mid 90s,"Jazzy Jungle, Jazzy DnB, Jazz n Bass, Sambaste...",jazzstep
99,Tech House,Minimal Tech,early 00s,"Plinky Plonk, Berliner School, Minimal Tech Ho...",minimaltech
93,Progressive,Progressive,mid 90s,Prog,progressive
27,Trance,Balearic Trance,early 90s,"Belearic House, Balearic Beat, Balearic Music,...",balearictrance
114,Breakbeat,Nu Skool Breaks,late 90s,"Tearout Breaks, Nu Breaks, NuSkool Breaks, Nu ...",nuskoolbreaks
124,Drum n Bass,Neurofunk,late 90s,"Electrostep, Psybreaks, Psychedelic Breaks",neurofunk
59,Chiptune,Fakebit,late 80s,"9-Bit, Riptune, Chiptune-inspired",fakebit
120,Drum n Bass,Jumpup,mid 90s,"Jump Up, Clownstep, SUUUUUUUUUUPER JUMPUP",jumpup
30,Psy Trance,Full On,late 90s,"Fullon Psy, Full-On, Uplifting Psy, Anthem Psy...",fullon
46,Intelligent Dance Music,Drill n Bass,mid 90s,Jungle IDM,drillnbass


In [388]:
df_genres.to_csv('../v3_genres.csv',index=False)